# Manipulating Molecules with RDKit

* How can I load files using RDKit?
* How can I manipulate and create new molecules from an existing molecule?

Load an "ideal" structure for a ligand obtained in the [EC_class_ligands_search](EC_class_ligands_search.ipynb) notebook by pulling coordinates from the PDB and then we manipulate it using RDKit.

The approach taken here is manual and hands on. Pick atoms of interest visually, then morph atomic numbers in order to change a carbon atom into a nitrogen atom, and then add a methyl group by changing a hydrogen atom into a carbon atom.

By loading molecule from an ideal structure with 3D coordinates, ensure near a "good structure" for manipulated molecules and geometry optimization will be more likely to succeed.



### Libraries for the IQB Workshop

| Library    | Description     |
| :-----------: | :------------ |
| rdkit | Cheminformatics Toolkit |
| Chem | A subset of rdkit for molecule manipulation |
| IPythonConsole | A subset of rdkit to control image quality |
| Draw | A subset of rdkit for structure drawing |
| AllChem | A subset of rdkit for optimizing 3D structures |

In [ ]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

# Configuration for displaying in Jupyter notebooks
IPythonConsole.ipython_useSVG = True  # Use SVG for higher quality images
IPythonConsole.drawOptions.addAtomIndices = True  # Show atom indices
IPythonConsole.molSize = 600,600 # Set size of image

ligand = Chem.MolFromMolFile("ligands/AZO_ideal.sdf")
ligand

### Modifying a ligand

Modify AZO from the previous notebook as starting point to create two additional ligands. In one case, will substitute nitrogen for carbon in an aromatic ring. In the second case, a carbon will be added to the original aromatic ring.

`IPythonConsole.drawOptions.addAtomIndices = True  # Show atom indices`

Will use these index numbers to tell the Python code which atoms to modify.

In [ ]:
# load a duplicate copy of 13U to manipulate
mod_ligand_N = Chem.MolFromMolFile("ligands/AZO_ideal.sdf")

# change carbon in ring to a nitrogen
mod_ligand_N.GetAtomWithIdx(23).SetAtomicNum(7)

mod_ligand_N

The nitrogen added has a hydrogen attached to it.
In order to maintain aromaticity, remove this hydrogen.

In [ ]:
atom = mod_ligand_N.GetAtomWithIdx(23) # Select our nitrogen atom
atom.SetNumExplicitHs(0) # Set the number of explicit hydrogens to 0

In [ ]:
# Look at position 23 now
mod_ligand_N

Perform the same operations, but this time add a methyl group to the ring.
Morph the hydrogen on the carbon atom to a carbon atom.
However, a more "standard" way to do this would be to create two molecule fragments and then merge them together.

In [ ]:
# load another duplicate of the original ligand, but keep the hydrogens

mod_ligand_methyl = Chem.MolFromMolFile("ligands/AZO_ideal.sdf", removeHs=False)
mod_ligand_methyl        # This is the original structure. In the cells below, we will convert Hydrogen-59 to a Carbon.

In [ ]:
# Use the index number to select the atom we want to change - look at image to see we want to morph atom 59
atom = mod_ligand_methyl.GetAtomWithIdx(59)

atom.SetAtomicNum(6) # Change the atom to carbon
atom.SetNumExplicitHs(3) # Add 3 explicit hydrogens to the carbon

mod_ligand_methyl = Chem.RemoveAllHs(mod_ligand_methyl) # Remove the H's for viewing
mod_ligand_methyl

The manipulated molecules, optimize them using RDKit and save them.

In [ ]:
# Optimize new molecules and save
from rdkit.Chem import AllChem

Chem.SanitizeMol(mod_ligand_N)
mod_ligand_NH = Chem.AddHs(mod_ligand_N)


# Do a constrained embedding to keep the ligand in the same position
# this allows for the hydrogens to be added in reasonable locations, but keeps
# the heavy atoms in the same position
# See https://rdkit.org/docs/source/rdkit.Chem.AllChem.html#rdkit.Chem.AllChem.ConstrainedEmbed
constrained_mol = AllChem.ConstrainedEmbed(mod_ligand_NH, mod_ligand_N, useTethers=True)
constrained_mol

In [ ]:
# Perform geometry optimization
opt_N = AllChem.MMFFOptimizeMolecule(mod_ligand_NH)
mod_ligand_NH

In [ ]:
# Repeat process on methyl ligand

Chem.SanitizeMol(mod_ligand_methyl)
mod_ligand_methylH = Chem.AddHs(mod_ligand_methyl)

constrained_mol = AllChem.ConstrainedEmbed(mod_ligand_methylH, mod_ligand_methyl, useTethers=True)
constrained_mol

In [ ]:
opt_methyl = AllChem.MMFFOptimizeMoleculeConfs(mod_ligand_methylH)
Chem.RemoveAllHs(mod_ligand_methylH)

In [ ]:
# save to new files
import os

# make modified ligand directory
os.makedirs("ligands_to_dock", exist_ok=True)

ligand_H = Chem.MolFromMolFile("ligands/13U_ideal.sdf", removeHs=False)

# save modified ligands sdf file - make sure all contain hydrogens and place
# in a folder of ligands to dock.
Chem.MolToMolFile(ligand_H, 'ligands_to_dock/13U.sdf')
Chem.MolToMolFile(mod_ligand_NH, 'ligands_to_dock/13U_modified_N.sdf')
Chem.MolToMolFile(mod_ligand_methylH, 'ligands_to_dock/13U_modified_methyl.sdf')

<div class="alert alert-block alert-warning">

<h3>Exercise</h3>

Take a different ligand from the ligands folder and define that as the ligand you wish to work with in the command

`ligand = Chem.MolFromMol2File("ligands/13U_ideal.mol2")`

Use the commands from this notebook to identify one hydrogen that you would like to change to a hydroxyl (-OH) group. Then make the change.
</div>

In [ ]:
# Solution using ligand 32U

ligand = Chem.MolFromMolFile("ligands/32U_ideal.sdf")
ligand

In [ ]:
mod_ligand_OH = Chem.MolFromMolFile("ligands/13U_ideal.sdf", removeHs=False)
mod_ligand_OH        # This is the original structure. In the cells below, we will convert Hydrogen-59 to a Carbon.

In [ ]:
# After inspecting the structure, I decided to add a phenolic hydroxyl to position 64

# Use the index number to select the atom we want to change - look at image to see we want to morph atom 64
atom = mod_ligand_OH.GetAtomWithIdx(64)

atom.SetAtomicNum(8) # Change the atom to oxygen
atom.SetNumExplicitHs(1) # Add 1 explicit hydrogen to the oxygen

mod_ligand_OH = Chem.RemoveAllHs(mod_ligand_OH) # Remove the H's for viewing
mod_ligand_OH